### Author: SHAAN ARYAMAN

In [1]:
import random

import numpy as np

import gurobipy as gp
from gurobipy import GRB
import pickle
import time
import copy
# import multiprocessing
# from multiprocessing import Pool

### Pickling

In [2]:
def read_pickle(filename):
    infile = open(filename,'rb')
    dict_data = pickle.load(infile)
    infile.close()
    return dict_data

In [3]:
def write_pickle(dict_data, filename):
    pickling_on = open(filename+'.pickle',"wb")
    pickle.dump(dict_data, pickling_on)
    pickling_on.close()
    print('saved file')

In [4]:
for i in [30, 40, 50, 60 , 70, 80, 90, 100, 110, 120]:
    file = 'data2/heuristic_results_size_' +str(i)+'.pickle'
    data = read_pickle(file)
    heuristic_run_time = data['heuristic_runtime']
    print(i, ' mean time limit: ', sum(heuristic_run_time.values())/len(heuristic_run_time.values()))
    print(i, " max run time: ", max(heuristic_run_time.values()))

30  mean time limit:  0.012791044623763473
30  max run time:  0.027321815490722656
40  mean time limit:  0.01953468068882271
40  max run time:  0.0428011417388916
50  mean time limit:  0.02729205279438584
50  max run time:  0.062166690826416016
60  mean time limit:  0.03647334641880459
60  max run time:  0.07808971405029297
70  mean time limit:  0.04627559593430272
70  max run time:  0.0941462516784668
80  mean time limit:  0.05985116285306436
80  max run time:  0.11557865142822266
90  mean time limit:  0.07363261922642036
90  max run time:  0.14253950119018555
100  mean time limit:  0.08989492533383546
100  max run time:  0.18332552909851074
110  mean time limit:  0.10854717360602485
110  max run time:  0.20955777168273926
120  mean time limit:  0.12676599389976925
120  max run time:  0.7496113777160645


In [5]:
file120 = 'data2/heuristic_results_size_120.pickle'
data120 = read_pickle(file)
heuristic_run_time120 = list(data['heuristic_runtime'].values())
heuristic_run_time120.sort(reverse=True)
print(heuristic_run_time120[:10])


[0.7496113777160645, 0.24187040328979492, 0.2370007038116455, 0.23236703872680664, 0.23151946067810059, 0.23040032386779785, 0.23017597198486328, 0.22837448120117188, 0.22834086418151855, 0.228102445602417]


### Gurobi Code

In [6]:
def run_gurobi(cloud_server, edge_server, tasks, V_k, U_j, C_k, 
               B_j, S_i, theta_i, G_i, tau_i, N_E_i, delta_jk, alpha, beta, time_limit, b_unit, c_unit):

    # Variables
    with gp.Env() as env, gp.Model(env=env) as model:
        
    # model = gp.Model('Cloud_Povisioning_0-1_ILP')

        model.Params.LogToConsole = 0

        model.Params.TimeLimit = time_limit

    #     Stop Gurobi from printing log data

        x_ijalpha = model.addVars(tasks, edge_server, alpha, vtype=GRB.BINARY, name='x_ijalpha')
        y_ikbeta = model.addVars(tasks, cloud_server+edge_server, beta, vtype=GRB.BINARY, name='y_ikbeta')
        z_ijk = model.addVars(tasks, edge_server, cloud_server+edge_server, vtype=GRB.BINARY, name='z_ijk')

    #     model.params.NonConvex = -1  # I think -1 is the default value

        # 10a
        for i in tasks:
            model.addConstr(

        gp.quicksum(x_ijalpha[i,j,a]*S_i[i] / (b_unit * a) for j in edge_server for a in alpha) +

        gp.quicksum(y_ikbeta[i,k,b]*theta_i[i] / (c_unit * b) for k in (cloud_server+edge_server) for b in beta) +

        2*gp.quicksum(z_ijk[i,j,k] * delta_jk[j,k] for j in edge_server for k in (cloud_server+edge_server))
                            <=tau_i[i], name= 'Deadline_const')

        # 10b
        for i in tasks:
            model.addConstr(gp.quicksum(x_ijalpha[i,j,a] for j in N_E_i[i] for a in alpha) <= 1, name='edge_server_allocation_1_const')

        # 10c
        for i in tasks:
            model.addConstr(gp.quicksum(x_ijalpha[i,j,a] for j in list(set(edge_server) - set(N_E_i[i])) for a in alpha) == 0, name='edge_server_allocation_2_const')

        # 10d
        for i in tasks:
            model.addConstr(gp.quicksum(y_ikbeta[i,k,b] for k in cloud_server+edge_server for b in beta) <= 1, name='cloud_server_allocation_const')

        # 10e
        for j in edge_server:
            model.addConstr(gp.quicksum(x_ijalpha[i,j,a] * a for i in tasks for a in alpha) <= U_j[j], name='bandwidth_capacity_const')

        # 10f
        for k in cloud_server+edge_server:
            model.addConstr(gp.quicksum(y_ikbeta[i,k,b] * b for i in tasks for b in beta) <= V_k[k], name='computation_capacity_const')

        # 10g
        for i in tasks:
            for j in edge_server:
                for k in cloud_server+edge_server:
                    model.addConstr(z_ijk[i,j,k] >= 0, name='z_positive_const')

        # 10h
        for i in tasks:
            for j in edge_server:
                for k in cloud_server+edge_server:
                    model.addConstr(z_ijk[i,j,k] >= gp.quicksum(x_ijalpha[i,j,a] for a in alpha)+
                                   gp.quicksum(y_ikbeta[i,k,b] for b in beta) -1, name = 'lower_bound_z_const')


        # 10i
        for i in tasks:
            for j in edge_server:
                for k in cloud_server+edge_server:
                    model.addConstr(z_ijk[i,j,k] <= gp.quicksum(x_ijalpha[i,j,a] for a in alpha), name='upper_bound_x_on_z_const')

        # 10j
        for i in tasks:
            for j in edge_server:
                for k in cloud_server+edge_server:
                    model.addConstr(z_ijk[i,j,k] <= gp.quicksum(y_ikbeta[i,k,b] for b in beta), name='upper_bound_y_on_z_const')

        obj = gp.quicksum(z_ijk[i,j,k] * G_i[i] for i in tasks for j in edge_server for k in cloud_server+edge_server)
        model.setObjective(obj, GRB.MAXIMIZE)

        model.optimize()

        '''
        Storing important variables from model run on one taskset instance
        obj_value: the profit value achieved
        x_ijalpha_stored_as_str_li: list of string containing task, edge server, alpha value
        y_ikbeta_stored_as_str_li: list of string containing task, edge/cloud server, beta value
        '''
        obj_value = model.getObjective().getValue()
        x_ijalpha_stored_as_str_li = []
        y_ikbeta_stored_as_str_li = []
        for key in x_ijalpha.keys():
            if x_ijalpha[key].x == 1:
                x_ijalpha_stored_as_str_li.append(x_ijalpha[key].VarName)
        for key in y_ikbeta.keys():
            if y_ikbeta[key].x == 1:
                y_ikbeta_stored_as_str_li.append(y_ikbeta[key].VarName)


    #     Resetting model
        model.dispose()
    
    return obj_value, x_ijalpha_stored_as_str_li, y_ikbeta_stored_as_str_li



In [7]:
def check_runtime_and_profit(tasksets, other_test_data, taskset_ID, time_limit):
    
    taskset = tasksets[taskset_ID]
    
    tasks = taskset['tasks']
    G_i = taskset['G_i']
    tau_i = taskset['tau_i']
    theta_i = taskset['theta_i']
    S_i = taskset['S_i']
    N_E_i = taskset['N_E_i']
    
    V_k = other_test_data['V_k']
    U_j = other_test_data['U_j']
    C_k = other_test_data['C_k']
    B_j = other_test_data['B_j']
    delta_jk = other_test_data['delta_jk']
    alpha = other_test_data['alpha']
    beta = other_test_data['beta']
    cloud_server = other_test_data['cloud_server']
    edge_server = other_test_data['edge_server']
    b_unit = other_test_data['unit_b']
    c_unit = other_test_data['unit_c']

    start_time = time.time()
    obj_value, x_ijalpha_stored_as_str_li, y_ikbeta_stored_as_str_li = run_gurobi(cloud_server, edge_server, tasks, V_k, U_j, C_k, 
               B_j, S_i, theta_i, G_i, tau_i, N_E_i, delta_jk, alpha, beta, time_limit, b_unit, c_unit)
    total_time_execution = time.time() - start_time
    
    return obj_value, total_time_execution, len(x_ijalpha_stored_as_str_li), len(y_ikbeta_stored_as_str_li)



In [8]:
def main(infile, outfile, time_limit, unit, start_id, end_id):
    dict_data = read_pickle(infile)

    other_test_data = dict_data['other_test_data']
    tasksets = dict_data['tasksets'] 
    results = dict_data['results'] 
    heuristic_profit = dict_data['heuristic_profit']
    # gurobi_time_limit = dict_data['gurobi_time_limit']
    heuristic_num_offloaded = dict_data['heuristic_num_offloaded']

    ###============================================###
    ###========= for large b_unit. c_unit =========###
    ###============================================###

    cloud_server_li = other_test_data['cloud_server']
    edge_server_li = other_test_data['edge_server']

    other_test_data['unit_b'] = unit
    other_test_data['unit_c'] = unit

    for j in other_test_data['edge_server']:
        other_test_data['U_j'][j] = other_test_data['U_j'][j] // other_test_data['unit_b']
    for k in cloud_server_li+edge_server_li:
        other_test_data['V_k'][k] = other_test_data['V_k'][k] // other_test_data['unit_c']

    max_Uj = max(other_test_data['U_j'].values())
    max_Vk = max(other_test_data['V_k'].values())

    other_test_data['alpha'] = []
    other_test_data['beta'] = []

    for a in range(1, 1+ max_Uj):
        other_test_data['alpha'].append(a)
    for b in range(1, 1+ max_Vk):
        other_test_data['beta'].append(b)
    ###============================================###


    print('Information from main()')
    print('-----------------------------------------------')
    print('Number of tasksets: ', len(tasksets.keys()))
    
    gurobi_profit, gurobi_runtime, gurobi_num_offloaded= {}, {}, {}

    ids = list(tasksets.keys())
    ids.sort()

    for ID in ids[start_id:end_id]:

        # time factor 200, threshold 3.12
        # time_limit = gurobi_time_limit[ID] * 2
        # if time_limit < 3.12:
        #     time_limit = 3.12


        obj_value, total_time_execution, num_offloaded_1, num_offloaded_2 = check_runtime_and_profit(tasksets, other_test_data, ID, time_limit)
        
        gurobi_profit[ID] = obj_value
        gurobi_runtime[ID] = total_time_execution
        gurobi_num_offloaded[ID] = num_offloaded_2
        
        
#         REMOVE this if condition
#         if((ID+1)%10 == 0):
#             break
        
        if((ID+1)%100 == 0):
            print('number of tasksets evaluated: ', ID+1)
          
    print('-----------------------------------------------')
    print()
    print()
    
    dict_data['gurobi_profit'] = gurobi_profit
    dict_data['gurobi_runtime'] = gurobi_runtime
    dict_data['gurobi_num_offloaded'] = gurobi_num_offloaded
    
    write_pickle(dict_data, outfile)
        
        
        

In [9]:
infile = 'data2/heuristic_results_size_110.pickle'
dict_data = read_pickle(infile)
print(dict_data.keys())
other_test_data = dict_data['other_test_data']
tasksets = dict_data['tasksets']
results = dict_data['results']
heuristic_profit = dict_data['heuristic_profit']
heuristic_run_time = list(dict_data['heuristic_runtime'].values())
heuristic_run_time.sort(reverse=True)
# gurobi_time_limit = dict_data['gurobi_time_limit']
heuristic_num_offloaded = dict_data['heuristic_num_offloaded']


# 80: 0, 1080,2160
# 90: 0, 1080,2160
# 100: 0, 720, 1440, 2160
# 110: 0, 720, 1440, 2160
# 120: 0, 720, 1440, 2160

running_time = heuristic_run_time[0] * 200
print("Gurobi Time Threshold: ", running_time)

start_id = 720
end_id = 1440

outfile = 'data2/gurobi_results_size_110_unit_5_2'
main(infile, outfile, running_time, 5, start_id, end_id)

# outfile = 'data2/gurobi_results_size_90_unit_10_1'
# main(infile, outfile, running_time, 10, start_id, end_id)

outfile = 'data2/gurobi_results_size_110_unit_15_2'
main(infile, outfile, running_time, 15, start_id, end_id)

dict_keys(['tasksets', 'results', 'heuristic_profit', 'heuristic_runtime', 'heuristic_num_offloaded', 'other_test_data'])
Gurobi Time Threshold:  41.91155433654785
Information from main()
-----------------------------------------------
Number of tasksets:  2160
Set parameter Username
Academic license - for non-commercial use only - expires 2023-04-01
Set parameter Username
Academic license - for non-commercial use only - expires 2023-04-01
Set parameter Username
Academic license - for non-commercial use only - expires 2023-04-01
Set parameter Username
Academic license - for non-commercial use only - expires 2023-04-01
Set parameter Username
Academic license - for non-commercial use only - expires 2023-04-01
Set parameter Username
Academic license - for non-commercial use only - expires 2023-04-01
Set parameter Username
Academic license - for non-commercial use only - expires 2023-04-01
Set parameter Username
Academic license - for non-commercial use only - expires 2023-04-01
Set parame

In [10]:
# def test(infile, time_factor):
#     dict_data = read_pickle(infile)
#
#     other_test_data = dict_data['other_test_data']
#     tasksets = dict_data['tasksets']
#     results = dict_data['results']
#     heuristic_profit = dict_data['heuristic_profit']
#     gurobi_time_limit = dict_data['gurobi_time_limit']
#     heuristic_num_offloaded = dict_data['heuristic_num_offloaded']
#
#     ###============================================###
#     ###========= for large b_unit. c_unit =========###
#     ###============================================###
#     unit = 5
#
#     cloud_server_li = other_test_data['cloud_server']
#     edge_server_li = other_test_data['edge_server']
#
#     other_test_data['unit_b'] = unit
#     other_test_data['unit_c'] = unit
#
#     for j in other_test_data['edge_server']:
#         other_test_data['U_j'][j] = other_test_data['U_j'][j] // other_test_data['unit_b']
#     for k in cloud_server_li+edge_server_li:
#         other_test_data['V_k'][k] = other_test_data['V_k'][k] // other_test_data['unit_c']
#
#     max_Uj = max(other_test_data['U_j'].values())
#     max_Vk = max(other_test_data['V_k'].values())
#
#     other_test_data['alpha'] = []
#     other_test_data['beta'] = []
#
#     for a in range(1, 1+ max_Uj):
#         other_test_data['alpha'].append(a)
#     for b in range(1, 1+ max_Vk):
#         other_test_data['beta'].append(b)
#     ###============================================###
#
#     print('Information from main()')
#     print('-----------------------------------------------')
#     print('Number of tasksets: ', len(tasksets.keys()))
#
#     gurobi_data = {}
#     gurobi_profit, gurobi_runtime, gurobi_num_offloaded = {}, {}, {}
#
#     samples = random.sample(list(tasksets.keys()), 10)
#
#     for ID in samples:
#         time_limit = gurobi_time_limit[ID]
#         obj_value, total_time_execution, num_offloaded_1, num_offloaded_2 = check_runtime_and_profit(tasksets, other_test_data, ID, time_limit * time_factor)
#         gurobi_profit[ID] = obj_value
#         gurobi_runtime[ID] = total_time_execution
#         gurobi_num_offloaded[ID] = num_offloaded_2
#
#     print('-----------------------------------------------')
#     print()
#     print()
#
#     gurobi_data['gurobi_profit'] = gurobi_profit
#     gurobi_data['gurobi_runtime'] = gurobi_runtime
#     gurobi_data['gurobi_num_offloaded'] = gurobi_num_offloaded
#     gurobi_data['sample_ids'] = samples
#
#     return gurobi_data
#
#
# for i in range(1, 5):
#     time_factor = i
#     gurobi_result = test(infile, time_factor)
#
#     print("Time Factor: ", time_factor * 100)
#     print("Task IDs: ", gurobi_result['sample_ids'])
#
#     print("Heuristic run time:")
#     for id in gurobi_result['sample_ids']:
#         print("{:.4f}".format(dict_data["gurobi_time_limit"][id] / 100), end=" ")
#     print()
#
#     print("Gurobi run time:")
#     for id in gurobi_result['sample_ids']:
#         print("{:.4f}".format(gurobi_result["gurobi_runtime"][id]), end=" ")
#     print()
#
#     print("Heuristic profit:")
#     for id in gurobi_result['sample_ids']:
#         print(dict_data["heuristic_profit"][id], end=" ")
#     print()
#
#     print("Gurobi profit:")
#     for id in gurobi_result['sample_ids']:
#         print(int(gurobi_result["gurobi_profit"][id]), end=" ")
#     print()
#
#     heuristic_mean = 0
#     for id in gurobi_result['sample_ids']:
#         heuristic_mean += dict_data["heuristic_profit"][id]
#     print("heuristic profit mean = ", heuristic_mean / len(gurobi_result['sample_ids']))
#
#     gurobi_mean = 0
#     for id in gurobi_result['sample_ids']:
#         gurobi_mean += int(gurobi_result["gurobi_profit"][id])
#     print("gurobi profit mean = ", gurobi_mean / len(gurobi_result['sample_ids']))
#
#     heuristic_mean_time = 0
#     for id in gurobi_result['sample_ids']:
#         heuristic_mean_time += dict_data["gurobi_time_limit"][id] / 100
#     print("heuristic mean time = ", heuristic_mean_time / len(gurobi_result['sample_ids']))
#
#     print("========================================")